In [5]:
#rm(list=ls())
library(magrittr)
library(dplyr)
#setwd("/home/huaqingj/MyResearch/HistTrial")
setwd("/home/r6user2/Documents/TQ/HistTrial")
#setwd("C:/Users/JINHU/Documents/ProjectCode/HistTrial")
source("utils.R")
library(parallel)

In [6]:
CI.fn <- function(errs){
  m.v <- mean(errs)
  sd.v <- sd(errs)
  se <- sd.v/sqrt(length(errs))
  low <- m.v - 1.96*se
  up <- m.v + 1.96*se
  rvs <- c(low, m.v, up)
  names(rvs) <- c("lower", "mean", "upper")
  rvs
}

### Test estimated trt eff

### Power size 

In [137]:
fun.test <- function(i){
    Xs <- gen.Data.Xs(n0, x.tps)
    idx0 <- sample.int(n0, size=floor(n0/2))
    Zs <- rep(1, n0)
    Zs[idx0] <- 0
    
    
    betMat <- sub.Paras.fn(Xs, betass)
    nerrs <- rnorm(n0, sd=phi0)
    Ys <- curMean.fn(Xs, Zs, betMat, b) + nerrs
    data <- cbind(Ys, Zs, Xs)
    data <- as.data.frame(data)
    colnames(data)[1:2] <- c("Y", "Z")
    
    
    # no borrowing 
    Zs.no <- Zs
    Ys.no <- curMean.fn(Xs, Zs.no, betMat, b) + nerrs
    data.no <- cbind(Ys.no, Zs.no, Xs)
    data.no <- as.data.frame(data.no)
    colnames(data.no)[1:2] <- c("Y", "Z")
    
    for (j in (n0+1):N){
        cx <- unlist(gen.Data.Xs(1, x.tps))
        
        # H <- diag(c(bw.nrd(data$X1), bw.nrd(data$X2), bw.nrd(data$X3), bw.nrd(data$X4)))
        alpMat <- sub.Paras.fn(Xs, alpss)
        Theta0s <- curMean.fn(Xs, Zs, alpMat, b=0)
        res <- info.est.fn(Theta0s, data, H, lam)
        res0 <- info.est.fn(Theta0s, data, H, lam, is.borrow=FALSE)
        
        var.info <- post.var.mu0.fn(cx, res)
        var.ref <- post.var.mu0.fn(cx, res0)
        R <- var.ref/var.info
        ass.res <- RPS.design(cx, data[, 3:6], data$Z, hs, R)
        ass.res.no <- RPS.design(cx, data[, 3:6], data.no$Z, hs, R=1)
        # ass.res <- RBC.design(cx, data[, 3:6], data$Z, hs, R)
        
        Xs <- rbind(Xs, cx)
        Zs <- c(Zs, ass.res$grp-1)
        Zs.no <- c(Zs.no, ass.res.no$grp-1)
        
        curN <- dim(Xs)[1]
        curBetMat <- sub.Paras.fn(Xs[curN, ], betass)
        nerr <- rnorm(1, sd=phi0)
        Y <- curMean.fn(Xs[curN, ], Zs[curN], curBetMat, b) + nerr
        Ys <- c(Ys, Y)
        data <- cbind(Ys, Zs, Xs)
        data <- as.data.frame(data)
        colnames(data)[1:2] <- c("Y", "Z")
        
        Y.no <- curMean.fn(Xs[curN, ], Zs.no[curN], curBetMat, b) + nerr
        Ys.no <- c(Ys.no, Y.no)
        data.no <- cbind(Ys.no, Zs.no, Xs)
        data.no <- as.data.frame(data.no)
        colnames(data.no)[1:2] <- c("Y", "Z")
        
        
    }
    
    alpMat <- sub.Paras.fn(Xs, alpss)
    Theta0s <- curMean.fn(Xs, Zs, alpMat, b=0)
    res <- info.est.fn(Theta0s, data, H, lam)
    res.no <- info.est.fn(Theta0s, data.no, H, lam, is.borrow=FALSE)
    
    res.mu1 <- mu1.est.fn(data$Y, data, H)
    res.no.mu1 <- mu1.est.fn(data.no$Y, data.no, H)
    trt.eff <- mean(mu1.efn(as.matrix(Xs), res.mu1)) -  mean(mu0.efn(as.matrix(Xs), res))
    trt.eff.no <- mean(mu1.efn(as.matrix(Xs), res.no.mu1)) -  mean(mu0.efn(as.matrix(Xs), res.no))
    post.prob.trt <- function(i){
        sps0 <- r.postMu0(as.matrix(Xs), res)
        sps0.no <- r.postMu0(as.matrix(Xs), res.no)
        sps1 <- r.postMu1(as.matrix(Xs), res.mu1)
        sps1.no <- r.postMu1(as.matrix(Xs), res.no.mu1)
        
        sps.trt <- mean(sps1) - mean(sps0)
        sps.trt.no <- mean(sps1.no) - mean(sps0.no)
        c(sps.trt, sps.trt.no)
    }
    sps.trts <- lapply(1:M, post.prob.trt)
    
    rv <- list(mtrt=c(trt.eff, trt.eff.no), sps.trts=sps.trts, data=data,data.no=data.no)
    rv
}

In [157]:
# parameters for current data 
betass <- list(
              para1=c(2, 1, -1, 3, -2), 
              para2=c(3, 1, 2, 4, 2), 
              para3=c(0, -1, -1, 3, 2), 
              para4=c(2, 0, -1, 4, -2)
                  )

# parameters for historical data
alpss <-  list(para1=c(2, 1, -1, 3, -2), 
             para2=c(3, 1, 2, 4, 2), 
             para3=c(0, -1, -1, 3, 2), 
             para4=c(2, 0, -1, 4, -2) )
b <- 0.5
phi0 = phi1 = 2
N <- 100 # total sample size
# parameters
lam <- 0.1
hs <- rep(2.1, 4)
x.tps <- c(2, 2, "c", "c")
#x.tps <- c(2, 2, 2, 2)

tXs <- gen.Data.Xs(1000, x.tps)
H <- diag(c(0.1, 0.1, 0.1, 0.1)/2)

# initial dataset
n0 <- 20

# for hypothesis test
# to calculate the prob
M <- 1000

In [158]:
nSimu <- 1000
post.res <- mclapply(1:nSimu, fun.test, mc.cores=60)
sv.name <- paste0("Res_test_b_", b, ".RData")
save(post.res, file=sv.name)

In [279]:
# for hypothesis test
dlt0 <- 1
# eps0 and eps0.no are tuned such that the size is 0.05 
eps0 <- 0.502
eps0.no <- 0.503
#eps0 <- 0.552
#eps0.no <- 0.543
# (n0, N, phi0) = (20, 50, 5): ep0s = (0.531, 0.527)
# (n0, N, phi0) = (20, 50, 10): ep0s = (0.529, 0.534)
# (n0, N, phi0) = (20, 100, 10): ep0s = (0.579, 0.571)
# (n0, N, phi0) = (20, 100, 2): ep0s = (0.519, 0.526)

In [280]:
post.fn <- function(trts){
    prob <- mean(trts > dlt0)
    prob
}

post.b.fn <- function(res){
    sps.trts <- res$sps.trts
    trtsMat <- do.call(rbind, sps.trts)
    prb <- post.fn(trtsMat[, 1])
    prb.no <- post.fn(trtsMat[, 2])
    c(prb, prb.no)
}

In [281]:
b = 0
c.res <- paste0("Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
    0.049     0.051

In [282]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.2865845,0.3009527,0.3153210
0.3420658,0.3591463,0.3762269


[1] 0.555

In [283]:
b = 0.5
c.res <- paste0("Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
    0.156     0.180

In [284]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.2842908,0.2980044,0.3117179
0.3570851,0.3740492,0.3910134


[1] 0.571

In [285]:
b = 1
c.res <- paste0("Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
    0.510     0.477

In [286]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.2830378,0.2973376,0.3116375
0.3411594,0.3573659,0.3735724


[1] 0.565

In [287]:
b = 1.5
c.res <- paste0("Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
    0.796     0.753

In [288]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.2953070,0.3104168,0.3255266
0.3536806,0.3706191,0.3875577


[1] 0.573

In [289]:
b = 2
c.res <- paste0("Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
    0.937     0.891

In [290]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.2876613,0.3020674,0.3164736
0.3336736,0.3505219,0.3673702


[1] 0.56

In [291]:
b = 2.5
c.res <- paste0("Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
    0.961     0.941

In [292]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.2800255,0.2944042,0.3087828
0.3547282,0.3717781,0.3888280


[1] 0.566

In [293]:
b = 3
c.res <- paste0("Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
    0.964     0.966

In [294]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.2986549,0.3129298,0.3272046
0.3454729,0.3628235,0.3801741


[1] 0.548

In [295]:
b = 4
c.res <- paste0("Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
     0.97      0.97

In [296]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.2913936,0.3057396,0.3200855
0.3403670,0.3570090,0.3736510


[1] 0.549

### Power size when beta is not equal to alp

In [355]:
# parameters for current data 
betass <- list(
              para1=c(2, 1, -1, 3, -2), 
              para2=c(3, 1, 2, 4, 2), 
              para3=c(0, -1, -1, 3, 2), 
              para4=c(2, 0, -1, 4, -2)
                  )

# parameters for historical data
alpss <-  list(para1=c(2, 1, -1, 3, -2), 
             para2=c(0, -1, -1, 3, 2), 
             para3=c(3, 1, 2, 4, 2), 
             para4=c(2, 0, -1, 4, -2) )
b <- 3.0
phi0 = phi1 = 2
N <- 100 # total sample size
# parameters
lam <- 0.1
hs <- rep(2.1, 4)
x.tps <- c(2, 2, "c", "c")
#x.tps <- c(2, 2, 2, 2)

tXs <- gen.Data.Xs(1000, x.tps)
H <- diag(c(0.1, 0.1, 0.1, 0.1)/2)

# initial dataset
n0 <- 20

# for hypothesis test
# to calculate the prob
M <- 1000

In [356]:
nSimu <- 1000
post.res <- mclapply(1:nSimu, fun.test, mc.cores=60)
sv.name <- paste0("Diff_Res_test_b_", b, ".RData")
save(post.res, file=sv.name)

In [378]:
# for hypothesis test
dlt0 <- 1
# eps0 and eps0.no are tuned such that the size is 0.05 
eps0 <- 0.510
eps0.no <- 0.506

In [379]:
post.fn <- function(trts){
    prob <- mean(trts > dlt0)
    prob
}

post.b.fn <- function(res){
    sps.trts <- res$sps.trts
    trtsMat <- do.call(rbind, sps.trts)
    prb <- post.fn(trtsMat[, 1])
    prb.no <- post.fn(trtsMat[, 2])
    c(prb, prb.no)
}

In [385]:
b = 3
c.res <- paste0("Diff_Res_test_b_", b, ".RData")
load(c.res)
probMat <- sapply(post.res, post.b.fn)
power <- mean(probMat[1, ] > eps0)
power.no <- mean(probMat[2,] > eps0.no)
powers <- c(power, power.no)
names(powers) <- c("Borrow", "No Borrow")
powers

Borrow No Borrow 
    0.901     0.944

In [381]:
trt.effs <- lapply(post.res, function(i)i$mtrt)
trt.effs.mat <- do.call(rbind, trt.effs)
errs <- abs(trt.effs.mat - b)
rbind(CI.fn(errs[, 1]), CI.fn(errs[, 2]))
mean(errs[, 1] < errs[, 2])

lower,mean,upper
0.3346243,0.3509774,0.3673304
0.3380078,0.3549688,0.3719298


[1] 0.509